# Introduction/Business problem

## My research has the following goal

### A friend of mine, who is in charge of a burger joint, needs to open a new location in Chicago. He intends to do it in a Douglas community area/borrough because he found numerous suitable places which are for sale or are available for rent. My friend is not very familiar with the city of Chicago and Douglas community area, which entails 8 neighbourhoods so he asked for help and needs me to analyze this community area and its most common places to decide which neighbourhood is the most suitable for the new location of his new burger place

### Since I am not a Chicago native and I'm not familiar with the Douglas community area, to help my friend I decided to use Foursquare loacation data and give him a report on the 8 neighbourhoods and their most common places, so we can decide which one is the most suitable for the new burger place

# Data section

### To solve our business problem I will have 2 main data sources in my report

## 1. Wikipedia article about the neighborhoods of Chicago

### The article has the following URL (https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago) <br>
### The article contains the table where the community areas with their respective neighborhoods are displayed. The table is a very suitable format since I can easily import it into my jupyter notebook and turn it into a pandas dataframe

### 2. Foursquare data <br>
### Second data source is of course Fousquare. I will connect to their database and get the data about the venues in the Douglas community area and its' neighborhoods. Then I will analyze it to display the most common places based on the neighborhoods and cluster these 8 neighborhoods into 5 clusters